In [1]:
# executeme

using NBInclude
@nbinclude("Hofstadter Single Particle in Julia.ipynb")

Hofstadter_SP (generic function with 1 method)

In [2]:
# executeme

#@time begin
using QuantumOptics

#https://juliapackages.com/p/einsum
using Einsum

# https://github.com/Jutho/TensorOperations.jl
using TensorOperations

using BenchmarkTools

using LinearAlgebra

#end

1) Hofstadter SP QoJulia energies are has to be equal to Hofstadter Single Particle energies.

In [7]:
Nx = 3; Ny = 3; N=Nx*Ny; q = Nx; N_cut = q
PN = 2
U = 2

2

In [13]:
basis = NLevelBasis(N)
Op = Hofstadter_SP(Nx,Ny,1/q, 0);

In [21]:
# executeme

function get_sp_op(basis, Op)
  
    H = SparseOperator(basis)

    for m in 1:N
        for n in 1:N
            H = H + Op[m,n] * transition(basis, m, n)
        end
    end
    
    return H
end

get_sp_op (generic function with 1 method)

In [22]:
H1 = get_sp_op(basis, Op)

Operator(dim=9x9)
  basis: NLevel(N=9)sparse([2, 3, 4, 7, 1, 3, 5, 8, 1, 2  …  8, 9, 2, 5, 7, 9, 3, 6, 7, 8], [1, 1, 1, 1, 2, 2, 2, 2, 3, 3  …  7, 7, 8, 8, 8, 8, 9, 9, 9, 9], ComplexF64[-1.0 + 0.0im, -1.0 + 0.0im, -1.0 + 0.0im, -1.0 + 0.0im, -1.0 + 0.0im, -1.0 + 0.0im, 0.4999999999999998 - 0.8660254037844387im, 0.4999999999999998 + 0.8660254037844387im, -1.0 + 0.0im, -1.0 + 0.0im  …  -1.0 + 0.0im, -1.0 + 0.0im, 0.4999999999999998 - 0.8660254037844387im, 0.4999999999999998 + 0.8660254037844387im, -1.0 + 0.0im, -1.0 + 0.0im, 0.5000000000000004 + 0.8660254037844386im, 0.5000000000000004 - 0.8660254037844386im, -1.0 + 0.0im, -1.0 + 0.0im], 9, 9)

In [23]:
#check op. form
eigenenergies(dense(H1)) == eigvals(Hofstadter_SP(Nx,Ny,1/q,0))

true

In [24]:
# executeme

function get_sub_states(SP_Op, N_cut)
    
    E0, states0 = eigenstates(dense(SP_Op))
    states = states0[1:N_cut]
    
    return states
end

get_sub_states (generic function with 1 method)

In [25]:
sub_states = get_sub_states(H1, N_cut);

In [26]:
# executeme

function get_Projector(states, Basis)
    
    b_sub = SubspaceBasis(Basis,states)
    P = projector(b_sub, Basis)
    Pt = dagger(P)
    
    return b_sub, P, Pt
end

get_Projector (generic function with 1 method)

In [27]:
b_sub, P, Pt = get_Projector(sub_states, basis);

In [30]:
# executeme

function Subspace_Op(SP_Op, P, Pt)
    return P*SP_Op*Pt
end

Subspace_Op (generic function with 1 method)

In [31]:
H1_sub = Subspace_Op(H1, P, Pt)

Operator(dim=3x3)
  basis: Subspace(superbasis=NLevel(N=9), states:3)
 -2.73205-2.0e-17im  -6.1e-16+1.9e-16im       0.0-5.0e-17im
 -6.7e-16-2.8e-16im  -2.73205-0.0im       7.0e-17-6.0e-17im
 -2.2e-16+3.0e-17im   6.0e-17+2.0e-17im  -2.73205-2.0e-17im

In [32]:
# executeme

function get_num_sub_list(basis,P,Pt)
    num_sub_list = []
    for m in 1:N
        NM = transition(basis, m, m)
        NMP = Subspace_Op(NM, P,Pt)
        push!(num_sub_list, NMP)
    end
    return num_sub_list
end

get_num_sub_list (generic function with 1 method)

In [33]:
num_sub_list = get_num_sub_list(basis,P,Pt);

In [34]:
states_mb = bosonstates(b_sub, PN) 
basis_mb = ManyBodyBasis(b_sub, states_mb)

ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=9), states:3), states:6)

In [35]:
# executeme

function get_MB_Op(MB_Basis, Basis, Op)
    
    Op_MB = SparseOperator(MB_Basis)
    for i in 1:length(Basis)
        for j in 1:length(Basis)
            Op_MB = Op_MB + Op.data[i,j] * transition(MB_Basis, i, j)
        end
    end
    
    return Op_MB
end

get_MB_Op (generic function with 1 method)

In [36]:
H1_MB = get_MB_Op(basis_mb, b_sub, H1_sub);

In [37]:
# H1_mb = manybodyoperator(basis_mb, H1_sub)

In [38]:
# executeme

function get_num_mb_list(basis_mb, b_sub, num_sub_list)
    num_mb_list = []
    for m in 1:N
        NMP = get_MB_Op(basis_mb, b_sub, num_sub_list[m])
        push!(num_mb_list, NMP)
    end
    return num_mb_list
end

get_num_mb_list (generic function with 1 method)

In [39]:
num_mb_list = get_num_mb_list(basis_mb, b_sub, num_sub_list);

In [40]:
function Int_Op(MB_Num_Op_List, MB_Basis, U)

    IT = SparseOperator(MB_Basis)
    
    for m in 1:N
        IT = IT + U/2 * ( MB_Num_Op_List[m]*MB_Num_Op_List[m] - MB_Num_Op_List[m] )
    end
    
    return IT
end

H_Int_MB = Int_Op(num_mb_list, basis_mb, U);

In [41]:
#@btime begin
@time begin
basis2 = basis ⊗ basis

# interaction : at_i at_i a_i a_i = at_i a_i at_i a_i - at_i a_i = n_i n_i - n_i
    
Vint = SparseOperator(basis2)

for n in 1:N
    Vint += U/2*transition(basis,n,n)⊗transition(basis,n,n)
end

Vint_sub = (P⊗P)*Vint*(Pt⊗Pt)

Vint_mb = manybodyoperator(basis_mb, Vint_sub)
end

  0.624291 seconds (773.67 k allocations: 40.553 MiB, 99.69% compilation time)


Operator(dim=6x6)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=9), states:3), states:6)
   0.325738-0.0im          0.0844895+0.0436973im   …    0.118168+0.0109049im
  0.0844895-0.0436973im     0.378162+0.0im            -0.0723289+0.0293332im
 -0.0260049-0.00677932im  -0.0607686-0.0546379im         2.0e-17+1.0e-17im
 -0.0352066+0.0497155im   -0.0444633+0.0229961im       0.0166959-0.0194952im
  0.0121908-0.00275542im   0.0367764+0.00958741im        3.0e-17-1.0e-17im
   0.118168-0.0109049im   -0.0723289-0.0293332im   …    0.416667+0.0im

In [42]:
P1 = P.data
P1t = Pt.data;

In [43]:
@time  @einsum P4[k,l,m,n] := P1[k,i] * P1[l,i] * P1t[i,m] * P1t[i,n];

  0.052721 seconds (23.46 k allocations: 1.135 MiB, 95.50% compilation time)


In [44]:
@time b2_sub = b_sub ⊗ b_sub

  0.000013 seconds (2 allocations: 176 bytes)


[Subspace(superbasis=NLevel(N=9), states:3) ⊗ Subspace(superbasis=NLevel(N=9), states:3)]

In [45]:
@time P4re = reshape(P4, N_cut^2, N_cut^2);

  0.010511 seconds (4.17 k allocations: 216.764 KiB, 99.04% compilation time)


In [46]:
@time Vint_bsub2 = SparseOperator(b2_sub,U/2*P4re);

  0.194335 seconds (414.87 k allocations: 21.131 MiB, 99.67% compilation time)


In [47]:
# executeme

function get_Vint_bsub2_einsum(P, Pt, b_sub, N_cut)
    P1 = P.data
    P1t = Pt.data;

    @einsum P4[k,l,m,n] := P1[k,i] * P1[l,i] * P1t[i,m] * P1t[i,n]

    b2_sub = b_sub ⊗ b_sub

    P4re = reshape(P4, N_cut^2, N_cut^2)

    Vint_bsub2 = SparseOperator(b2_sub,U/2*P4re)
    #Vint_bsub2 = Operator(b2_sub,U/2*P4re)

    return Vint_bsub2
end

get_Vint_bsub2_einsum (generic function with 1 method)

In [48]:
@time begin
Vint_bsub2 = get_Vint_bsub2_einsum(P, Pt, b_sub, N_cut)
end;

  0.229657 seconds (478.47 k allocations: 26.829 MiB, 99.97% compilation time)


In [49]:
@time begin
Vint_mb_einsum = manybodyoperator(basis_mb, Vint_bsub2)
end;

  0.233459 seconds (486.67 k allocations: 23.686 MiB, 99.89% compilation time)


In [50]:
function manybodyoperator_2_Hubbard(basis::ManyBodyBasis, op::SparseOpType)
    N = length(basis)
    S = length(basis.onebodybasis)
    result = SparseOperator(basis)
    occupations = basis.occupations
    rows = QuantumOpticsBase.rowvals(op.data)
    values = QuantumOpticsBase.nonzeros(op.data)
    @inbounds for column=1:S^2, j in QuantumOpticsBase.nzrange(op.data, column)
        row = rows[j]
        value = values[j]
        index = Tuple(CartesianIndices((S, S, S, S))[(column-1)*S^2 + row])
        for m=1:N, n=1:N
            # println("row:", row, " column:"column, ind_left)
            C = QuantumOpticsBase.coefficient(occupations[m], occupations[n], index[1:2], index[3:4])
            if C!=0.
                result.data[m,n] += C*value
            end
        end
    end
    return result
end

manybodyoperator_2_Hubbard (generic function with 1 method)

In [51]:
@time Vint_mb_Hubbard = manybodyoperator_2_Hubbard(basis_mb, Vint_bsub2);

  0.079633 seconds (77.51 k allocations: 4.080 MiB, 99.19% compilation time)


In [52]:
isapprox( Vint_mb_einsum, Vint_mb_Hubbard )

true

In [53]:
isapprox( Vint_mb , Vint_mb_einsum )

true

In [54]:
Vint_mb_einsum.data

6×6 SparseArrays.SparseMatrixCSC{ComplexF64, Int64} with 36 stored entries:
   0.325738+5.85469e-18im  …     0.118168+0.0109049im
  0.0844895-0.0436973im        -0.0723289+0.0293332im
 -0.0260049-0.00677932im      2.94392e-17-2.14661e-18im
 -0.0352066+0.0497155im         0.0166959-0.0194952im
  0.0121908-0.00275542im      3.92523e-17-2.08528e-17im
   0.118168-0.0109049im    …     0.416667+0.0im

In [55]:
Vint_mb_Hubbard.data

6×6 SparseArrays.SparseMatrixCSC{ComplexF64, Int64} with 36 stored entries:
   0.325738+5.85469e-18im  …     0.118168+0.0109049im
  0.0844895-0.0436973im        -0.0723289+0.0293332im
 -0.0260049-0.00677932im      2.94392e-17-2.14661e-18im
 -0.0352066+0.0497155im         0.0166959-0.0194952im
  0.0121908-0.00275542im      3.92523e-17-2.08528e-17im
   0.118168-0.0109049im    …     0.416667+0.0im

In [56]:
H_MB = H1_MB + Vint_mb_einsum
H_mb = H1_MB + Vint_mb_Hubbard

Operator(dim=6x6)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=9), states:3), states:6)sparse([1, 2, 3, 4, 5, 6, 1, 2, 3, 4  …  3, 4, 5, 6, 1, 2, 3, 4, 5, 6], [1, 1, 1, 1, 1, 1, 2, 2, 2, 2  …  5, 5, 5, 5, 6, 6, 6, 6, 6, 6], ComplexF64[-5.138363413839574 - 4.0e-17im, 0.08448947538935428 - 0.04369730616738181im, -0.02600487162459144 - 0.00677931948613955im, -0.03520664040389488 + 0.04971550027806915im, 0.01219083708705511 - 0.00275541629694062im, 0.11816805340776146 - 0.01090490889120976im, 0.08448947538935435 + 0.04369730616738169im, -5.085939843890932 - 3.0e-17im, -0.06076863193762786 - 0.05463787069570918im, -0.04446328505092807 + 0.02299606869523398im  …  -0.05660558122673197 - 0.02927597079101634im, 0.04296991172652607 - 0.03863480887852963im, -5.2677967746278975 - 3.0e-17im, 9.0e-17 + 3.0e-17im, 0.11816805340776149 + 0.01090490889120977im, -0.07232892961281417 + 0.02933321289465911im, 3.0e-17 - 7.0e-17im, 0.01669592688763701 - 0.0194951798384127im, 1.4e-16 - 1.0e-16i

2) If U<<Band-Gap, Sub-Space Hofstadter Finite-U energies converges at Hofstadter Finite-U energies.
3) When all SP states projected to the new sub-space, Sub-Space Hofstadter Finite-U energies have to equal to Hofstadter Finite-U energies.

In [57]:
@nbinclude("Hofstadter MB in Julia.ipynb"; regex=r"#.*executeme")

Hofstadter_Finite_U (generic function with 1 method)

In [58]:
# using Plots

E1 = eigenenergies(dense((H_MB+dagger(H_MB))/2))

E_mb = eigenenergies(dense((H_mb+dagger(H_mb))/2))


E2 = eigenenergies(dense(Hofstadter_Finite_U(Nx, Ny, 1/q, PN, U)))

# print(E1-E2[1:length(E1)])

# plot(1:length(E1),E1,seriestype=:scatter,markershape=:star5, markersize=6, label="Sub-Space")
# plot!(1:length(E2),E2,seriestype=:scatter,label="Real-Space",legend=:topleft)
# xlabel!("n");ylabel!("E")

print(E1,'\n', E_mb,'\n',E2[1:10])

[-5.297434948471096, -5.29743494847109, -5.2974349484710865, -4.964101615137758, -4.964101615137757, -4.9641016151377535]
[-5.297434948471096, -5.29743494847109, -5.2974349484710865, -4.964101615137758, -4.964101615137757, -4.9641016151377535]
[-5.340971176014525, -5.340971176014523, -5.34097117601451, -5.066168265618834, -5.066168265618833, -5.066168265618821, -1.8128503544788694, -1.812850354478869, -1.8128503544788679, -1.704570336931171]